In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
%matplotlib inline

In [5]:
df = pd.read_csv('analysisData.csv')
sub = pd.read_csv('scoringData.csv')
df = pd.concat([df,sub])

In [6]:
pd.set_option('display.max_columns', None)
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,20091785,https://www.airbnb.com/rooms/20091785,20180303203649,2018-03-05,"Cozy Stay in Queens, Easy Access to Manhattan","A perfect spot for an intern, student or out o...",NaN,"A perfect spot for an intern, student or out o...",none,We're right next to the best Indian food in Qu...,NaN,"Parking is limited, but we're near the E/F/M/R...","You're welcome to chill in your room, cozy up ...",I am available via text or email. I'm happy to...,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/438180da-d...,NaN,142871086,https://www.airbnb.com/users/show/142871086,Emily,2017-07-27,"New York, New York, United States",I am a published playwright and novelist livin...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/fbfde7c2-f...,https://a0.muscache.com/im/pictures/fbfde7c2-f...,Jackson Heights,1,1,"['email', 'phone', 'facebook', 'google', 'revi...",t,f,"Queens, NY, United States",Jackson Heights,East Elmhurst,Queens,Queens,NY,11370,New York,"Queens, NY",US,United States,40.754444,-73.895933,t,Apartment,Private room,1,1.0,1,1.0,Real Bed,"{Wifi,Kitchen,Heating,""Smoke detector"",""Carbon...",NaN,45,NaN,NaN,NaN,NaN,1,0,1,1125,7 months ago,t,0,0,0,0,2018-03-05,3,2017-08-02,2017-08-03,90,9,8,10,10,10,10,f,NaN,NaN,t,f,flexible,f,f,1,0.42
1,3710661,https://www.airbnb.com/rooms/3710661,20180303203649,2018-03-04,Spacious room in comfortable apt.,"Nice size apt 15 minutes from central park, a...",This is a private room in a two bedroom apartm...,"Nice size apt 15 minutes from central park, a...",none,The apartment is conveniently located on the b...,Apartment located on border UES and East H.N...,You can take the 6 train downtown and get to T...,Guests should take there shoes off at the fron...,I can assist with any question you may have re...,I am asking you to follow these few basic rule...,NaN,NaN,https://a0.muscache.com/im/pictures/57097196/5...,NaN,18930170,https://www.airbnb.com/users/show/18930170,Tom,2014-07-26,"New York, New York, United States","I have been living in New York for 9years, I l...",within an hour,100%,NaN,t,https://a0.muscache.com/im/pictures/6b4dfca5-4...,https://a0.muscache.com/im/pictures/6b4dfca5-4...,East Harlem,3,3,"['email', 'phone', 'reviews', 'kba']",t,t,"New York, NY, United States",East Harlem,East Harlem,Manhattan,New York,NY,10029,New York,"New York, NY",US,United States,40.785097,-73.942337,f,Apartment,Private room,2,1.0,1,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",NaN,95,NaN,NaN,NaN,50.0,2,0,2,99,3 days ago,t,3,8,24,74,2018-03-04,101,2014-08-11,2018-01-20,93,10,9,10

In [296]:
df.drop(['id', 'experiences_offered','listing_url', 'scrape_id','name', 'summary','space','description','thumbnail_url','medium_url','picture_url',
         'xl_picture_url','host_id','host_url','host_name','host_location','host_thumbnail_url','host_picture_url','host_neighbourhood',
        'street','neighbourhood','neighbourhood_group_cleansed','city','state','market','smart_location','country_code','latitude',
        'longitude','country', 'zipcode'], axis=1, inplace=True)

In [297]:
df.shape

(36428, 65)

### Text columns : T/F based on NaN

In [298]:
str_list = ['neighborhood_overview','notes','transit','access','interaction','house_rules','host_about']

for var in str_list:
    df[var] = df[var].apply(lambda x: 1 if x is np.nan else 0)

### Missing Value over 70%

In [9]:
pd.set_option('display.max_rows', None)
na_df = pd.DataFrame(df.isna().sum()/df.shape[0])

In [300]:
df.drop(list(na_df[na_df[0] > 0.7].index), axis=1, inplace=True)

### Unixtime

In [301]:
import datetime
time_list = ['last_scraped','host_since','calendar_last_scraped']
for var in time_list:
    df[var] = df[var].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').timestamp()).astype(int)

In [302]:
review_list = ['first_review','last_review']    
for var in review_list:
    df[var] = df[var].apply(lambda x: datetime.datetime.strptime(str(x), '%Y-%m-%d').timestamp() if x is not np.NaN else x)

### Oridnal Categoty to Int

In [303]:
def res(x):
    if x =='within an hour':
        return 0
    elif x == 'within a few hours':
        return 1
    elif x == 'within a day':
        return 2
    elif x == 'a few days or more':
        return 3
    else: 
        return x

In [304]:
df['host_response_time'] = df['host_response_time'].apply(lambda x: res(x))

In [305]:
def can_pol(x):
    if x =='flexible':
        return 0
    elif x == 'moderate':
        return 1
    elif x == 'strict':
        return 2
    elif x == 'super_strict_30':
        return 3
    elif x == 'super_strict_60':
        return 3
    else: 
        return x

In [306]:
df['cancellation_policy'] = df['cancellation_policy'].apply(lambda x: can_pol(x))

### Other String Columns

In [307]:
# Remove %
df['host_response_rate'] = df['host_response_rate'].str.strip('%').astype(float)
# df['host_response_rate'] = df['host_response_rate'].astype(float)

In [14]:
# Convert string to int: calnerdar_undated 
cal = pd.DataFrame(list(df['calendar_updated'].apply(lambda x: x.split())), columns=['num','scale','nn'])

In [15]:
def cal_num(x):
    if x =='yesterday':
        return 1
    elif x == 'today':
        return 0
    elif x == 'a':
        return 1
    elif x == 'never':
        return 1000     #I checked a distplot and assigned reasonable number which is high enough
    else: 
        return x

In [16]:
cal['num'] = cal['num'].apply(lambda x: cal_num(x))

In [17]:
def cal_scale(x):
    if x =='months':
        return 30
    elif x == 'days':
        return 1
    elif x == 'week':
        return 7
    elif x == 'weeks':
        return 7
    else: 
        return 1

In [18]:
cal['scale'] = cal['scale'].apply(lambda x: cal_scale(x))

In [19]:
cal['num'] = cal['num'].astype(float)

In [20]:
cal['sum'] = cal['num'] * cal['scale']

In [21]:
df['calendar_updated'] = cal['sum']

In [316]:
# Convert True/False columns
def tf(x):
    if x =='t':
        return 1
    elif x == 'f':
        return 0
    else: 
        return x

In [317]:
tf_list = ['host_is_superhost','host_has_profile_pic','host_identity_verified','is_location_exact','has_availability','requires_license',
          'instant_bookable','is_business_travel_ready','require_guest_profile_picture','require_guest_phone_verification']

In [318]:
for var in tf_list:
    df[var] = df[var].apply(lambda x: tf(x))

In [24]:
# Split List Columns: host_verifications
import re
df['host_verifications'] = df['host_verifications'].apply(lambda x: re.sub('[^A-Za-z0-9,]+', '', x).split(','))

In [26]:
veri = df['host_verifications']
veri_list = []
for i in range(len(veri)):
    veri_list += veri.iloc[i]
    veri_list = list(set(veri_list))    

In [28]:
for i in veri_list:
    df[i] = df['host_verifications'].apply(lambda x: 1 if i in x else 0)

In [29]:
df.drop('host_verifications',axis=1, inplace=True)

In [323]:
# Split List Columns: amenity
df['amenities'] = df['amenities'].apply(lambda x: re.sub('[^A-Za-z0-9,]+', '', x).split(','))

amen = df['amenities']
amen_list = []
for i in range(len(amen)):
    amen_list += amen.iloc[i]
    amen_list = list(set(amen_list)) 

amen_list.remove('')

for i in amen_list:
    df[i] = df['amenities'].apply(lambda x: 1 if i in x else 0)    

df.drop('amenities',axis=1, inplace=True)

### Get Dummy

In [324]:
df = pd.get_dummies(df, columns = ['neighbourhood_cleansed','property_type','room_type','bed_type'], drop_first=True)

In [328]:
df.drop('', axis=1, inplace=True)

### Split data

In [329]:
sub = df[pd.isnull(df['price'])]
df = df[~pd.isnull(df['price'])]

In [330]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('price',axis=1), df['price'], test_size=0.3, random_state=83)

### LGBM

In [248]:
import lightgbm

In [331]:
train_data = lightgbm.Dataset(X_train, label=y_train)
test_data = lightgbm.Dataset(X_test, label=y_test)

In [352]:
parameters = {
    'objective': 'tweedie',
    'metric': 'rmse',
    'num_leaves': 15,
    'feature_fraction': 0.3,
    'bagging_fraction': 0.3,
    'bagging_freq': 20,
    'learning_rate': 0.09,
    'verbose': 100,
    'max_depth': -1,
    'lambda_l1' : 0.39 ,
    'lambda_l2' : 0.7,
    'tweedie_variance_power': 1.6
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=1000)

[1]	valid_0's rmse: 105.801
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's rmse: 103.6
[3]	valid_0's rmse: 101.098
[4]	valid_0's rmse: 99.2903
[5]	valid_0's rmse: 96.8254
[6]	valid_0's rmse: 95.5145
[7]	valid_0's rmse: 93.7208
[8]	valid_0's rmse: 92.3464
[9]	valid_0's rmse: 91.001
[10]	valid_0's rmse: 89.6243
[11]	valid_0's rmse: 88.1296
[12]	valid_0's rmse: 87.0251
[13]	valid_0's rmse: 85.2725
[14]	valid_0's rmse: 84.2815
[15]	valid_0's rmse: 84.0242
[16]	valid_0's rmse: 82.9517
[17]	valid_0's rmse: 82.5814
[18]	valid_0's rmse: 81.5828
[19]	valid_0's rmse: 80.7868
[20]	valid_0's rmse: 80.3518
[21]	valid_0's rmse: 79.7447
[22]	valid_0's rmse: 79.202
[23]	valid_0's rmse: 78.0693
[24]	valid_0's rmse: 77.8123
[25]	valid_0's rmse: 77.1195
[26]	valid_0's rmse: 76.3179
[27]	valid_0's rmse: 75.9507
[28]	valid_0's rmse: 75.423
[29]	valid_0's rmse: 74.8458
[30]	valid_0's rmse: 74.3954
[31]	valid_0's rmse: 73.6978
[32]	valid_0's rmse: 73.2577
[33]	valid_0's rmse: 7

[352]	valid_0's rmse: 60.0961
[353]	valid_0's rmse: 60.0813
[354]	valid_0's rmse: 60.0827
[355]	valid_0's rmse: 60.0757
[356]	valid_0's rmse: 60.0819
[357]	valid_0's rmse: 60.0587
[358]	valid_0's rmse: 60.0629
[359]	valid_0's rmse: 60.0601
[360]	valid_0's rmse: 60.0584
[361]	valid_0's rmse: 60.0584
[362]	valid_0's rmse: 60.0541
[363]	valid_0's rmse: 60.025
[364]	valid_0's rmse: 60.0213
[365]	valid_0's rmse: 60.0182
[366]	valid_0's rmse: 60.0117
[367]	valid_0's rmse: 60.0132
[368]	valid_0's rmse: 59.9624
[369]	valid_0's rmse: 59.9651
[370]	valid_0's rmse: 59.9412
[371]	valid_0's rmse: 59.9348
[372]	valid_0's rmse: 59.9294
[373]	valid_0's rmse: 59.9111
[374]	valid_0's rmse: 59.9003
[375]	valid_0's rmse: 59.8952
[376]	valid_0's rmse: 59.8897
[377]	valid_0's rmse: 59.887
[378]	valid_0's rmse: 59.8643
[379]	valid_0's rmse: 59.8624
[380]	valid_0's rmse: 59.8619
[381]	valid_0's rmse: 59.8463
[382]	valid_0's rmse: 59.836
[383]	valid_0's rmse: 59.8426
[384]	valid_0's rmse: 59.8565
[385]	valid_0

[642]	valid_0's rmse: 59.2703
[643]	valid_0's rmse: 59.2696
[644]	valid_0's rmse: 59.2811
[645]	valid_0's rmse: 59.2817
[646]	valid_0's rmse: 59.2733
[647]	valid_0's rmse: 59.2639
[648]	valid_0's rmse: 59.2608
[649]	valid_0's rmse: 59.2709
[650]	valid_0's rmse: 59.2627
[651]	valid_0's rmse: 59.2495
[652]	valid_0's rmse: 59.2441
[653]	valid_0's rmse: 59.2481
[654]	valid_0's rmse: 59.2538
[655]	valid_0's rmse: 59.2539
[656]	valid_0's rmse: 59.2508
[657]	valid_0's rmse: 59.2523
[658]	valid_0's rmse: 59.2535
[659]	valid_0's rmse: 59.2552
[660]	valid_0's rmse: 59.2567
[661]	valid_0's rmse: 59.2569
[662]	valid_0's rmse: 59.2584
[663]	valid_0's rmse: 59.2628
[664]	valid_0's rmse: 59.2646
[665]	valid_0's rmse: 59.2622
[666]	valid_0's rmse: 59.246
[667]	valid_0's rmse: 59.241
[668]	valid_0's rmse: 59.2401
[669]	valid_0's rmse: 59.2436
[670]	valid_0's rmse: 59.2369
[671]	valid_0's rmse: 59.2344
[672]	valid_0's rmse: 59.2365
[673]	valid_0's rmse: 59.2381
[674]	valid_0's rmse: 59.2369
[675]	valid_

[967]	valid_0's rmse: 58.9202
[968]	valid_0's rmse: 58.9098
[969]	valid_0's rmse: 58.9106
[970]	valid_0's rmse: 58.9051
[971]	valid_0's rmse: 58.9076
[972]	valid_0's rmse: 58.9158
[973]	valid_0's rmse: 58.9165
[974]	valid_0's rmse: 58.918
[975]	valid_0's rmse: 58.9197
[976]	valid_0's rmse: 58.9109
[977]	valid_0's rmse: 58.914
[978]	valid_0's rmse: 58.9142
[979]	valid_0's rmse: 58.918
[980]	valid_0's rmse: 58.9167
[981]	valid_0's rmse: 58.9149
[982]	valid_0's rmse: 58.913
[983]	valid_0's rmse: 58.9129
[984]	valid_0's rmse: 58.9105
[985]	valid_0's rmse: 58.9115
[986]	valid_0's rmse: 58.9067
[987]	valid_0's rmse: 58.9073
[988]	valid_0's rmse: 58.9071
[989]	valid_0's rmse: 58.906
[990]	valid_0's rmse: 58.905
[991]	valid_0's rmse: 58.8938
[992]	valid_0's rmse: 58.894
[993]	valid_0's rmse: 58.8879
[994]	valid_0's rmse: 58.8894
[995]	valid_0's rmse: 58.8888
[996]	valid_0's rmse: 58.8915
[997]	valid_0's rmse: 58.8916
[998]	valid_0's rmse: 58.8892
[999]	valid_0's rmse: 58.8912
[1000]	valid_0's 

[1259]	valid_0's rmse: 58.6622
[1260]	valid_0's rmse: 58.6619
[1261]	valid_0's rmse: 58.6638
[1262]	valid_0's rmse: 58.6633
[1263]	valid_0's rmse: 58.6615
[1264]	valid_0's rmse: 58.6626
[1265]	valid_0's rmse: 58.6641
[1266]	valid_0's rmse: 58.6732
[1267]	valid_0's rmse: 58.6748
[1268]	valid_0's rmse: 58.6786
[1269]	valid_0's rmse: 58.6826
[1270]	valid_0's rmse: 58.6864
[1271]	valid_0's rmse: 58.6862
[1272]	valid_0's rmse: 58.6887
[1273]	valid_0's rmse: 58.6896
[1274]	valid_0's rmse: 58.691
[1275]	valid_0's rmse: 58.6958
[1276]	valid_0's rmse: 58.6868
[1277]	valid_0's rmse: 58.686
[1278]	valid_0's rmse: 58.6879
[1279]	valid_0's rmse: 58.6954
[1280]	valid_0's rmse: 58.6949
[1281]	valid_0's rmse: 58.7072
[1282]	valid_0's rmse: 58.7055
[1283]	valid_0's rmse: 58.704
[1284]	valid_0's rmse: 58.704
[1285]	valid_0's rmse: 58.7149
[1286]	valid_0's rmse: 58.7105
[1287]	valid_0's rmse: 58.7084
[1288]	valid_0's rmse: 58.7067
[1289]	valid_0's rmse: 58.7111
[1290]	valid_0's rmse: 58.7205
[1291]	valid

...

[3564]	valid_0's rmse: 57.8808
[3565]	valid_0's rmse: 57.8785
[3566]	valid_0's rmse: 57.8784
[3567]	valid_0's rmse: 57.8777
[3568]	valid_0's rmse: 57.88
[3569]	valid_0's rmse: 57.8801
[3570]	valid_0's rmse: 57.8813
[3571]	valid_0's rmse: 57.8832
[3572]	valid_0's rmse: 57.8858
[3573]	valid_0's rmse: 57.885
[3574]	valid_0's rmse: 57.8852
[3575]	valid_0's rmse: 57.8879
[3576]	valid_0's rmse: 57.8889
[3577]	valid_0's rmse: 57.8888
[3578]	valid_0's rmse: 57.8898
[3579]	valid_0's rmse: 57.8893
[3580]	valid_0's rmse: 57.8897
[3581]	valid_0's rmse: 57.889
[3582]	valid_0's rmse: 57.8846
[3583]	valid_0's rmse: 57.8824
[3584]	valid_0's rmse: 57.8812
[3585]	valid_0's rmse: 57.8786
[3586]	valid_0's rmse: 57.8753
[3587]	valid_0's rmse: 57.8745
[3588]	valid_0's rmse: 57.8685
[3589]	valid_0's rmse: 57.8635
[3590]	valid_0's rmse: 57.8601
[3591]	valid_0's rmse: 57.853
[3592]	valid_0's rmse: 57.8533
[3593]	valid_0's rmse: 57.8527
[3594]	valid_0's rmse: 57.8513
[3595]	valid_0's rmse: 57.8518
[3596]	valid_

[3832]	valid_0's rmse: 57.8274
[3833]	valid_0's rmse: 57.8267
[3834]	valid_0's rmse: 57.829
[3835]	valid_0's rmse: 57.8258
[3836]	valid_0's rmse: 57.8248
[3837]	valid_0's rmse: 57.8242
[3838]	valid_0's rmse: 57.8273
[3839]	valid_0's rmse: 57.8265
[3840]	valid_0's rmse: 57.829
[3841]	valid_0's rmse: 57.8264
[3842]	valid_0's rmse: 57.8275
[3843]	valid_0's rmse: 57.8271
[3844]	valid_0's rmse: 57.8213
[3845]	valid_0's rmse: 57.8215
[3846]	valid_0's rmse: 57.8251
[3847]	valid_0's rmse: 57.8282
[3848]	valid_0's rmse: 57.8277
[3849]	valid_0's rmse: 57.8281
[3850]	valid_0's rmse: 57.8259
[3851]	valid_0's rmse: 57.8153
[3852]	valid_0's rmse: 57.8162
[3853]	valid_0's rmse: 57.8156
[3854]	valid_0's rmse: 57.815
[3855]	valid_0's rmse: 57.8013
[3856]	valid_0's rmse: 57.8012
[3857]	valid_0's rmse: 57.8013
[3858]	valid_0's rmse: 57.7946
[3859]	valid_0's rmse: 57.7948
[3860]	valid_0's rmse: 57.7947
[3861]	valid_0's rmse: 57.7958
[3862]	valid_0's rmse: 57.7954
[3863]	valid_0's rmse: 57.7943
[3864]	vali

[4142]	valid_0's rmse: 57.7268
[4143]	valid_0's rmse: 57.7265
[4144]	valid_0's rmse: 57.7244
[4145]	valid_0's rmse: 57.7274
[4146]	valid_0's rmse: 57.7302
[4147]	valid_0's rmse: 57.7307
[4148]	valid_0's rmse: 57.7318
[4149]	valid_0's rmse: 57.7322
[4150]	valid_0's rmse: 57.7325
[4151]	valid_0's rmse: 57.7325
[4152]	valid_0's rmse: 57.7351
[4153]	valid_0's rmse: 57.7352
[4154]	valid_0's rmse: 57.7356
[4155]	valid_0's rmse: 57.7361
[4156]	valid_0's rmse: 57.7358
[4157]	valid_0's rmse: 57.7346
[4158]	valid_0's rmse: 57.7359
[4159]	valid_0's rmse: 57.7332
[4160]	valid_0's rmse: 57.7329
[4161]	valid_0's rmse: 57.7338
[4162]	valid_0's rmse: 57.7348
[4163]	valid_0's rmse: 57.7341
[4164]	valid_0's rmse: 57.7339
[4165]	valid_0's rmse: 57.7339
[4166]	valid_0's rmse: 57.7348
[4167]	valid_0's rmse: 57.735
[4168]	valid_0's rmse: 57.7344
[4169]	valid_0's rmse: 57.7348
[4170]	valid_0's rmse: 57.732
[4171]	valid_0's rmse: 57.7329
[4172]	valid_0's rmse: 57.7298
[4173]	valid_0's rmse: 57.7302
[4174]	val

[4429]	valid_0's rmse: 57.7385
[4430]	valid_0's rmse: 57.7392
[4431]	valid_0's rmse: 57.7428
[4432]	valid_0's rmse: 57.7425
[4433]	valid_0's rmse: 57.7422
[4434]	valid_0's rmse: 57.7409
[4435]	valid_0's rmse: 57.7431
[4436]	valid_0's rmse: 57.7429
[4437]	valid_0's rmse: 57.7434
[4438]	valid_0's rmse: 57.743
[4439]	valid_0's rmse: 57.7356
[4440]	valid_0's rmse: 57.7352
[4441]	valid_0's rmse: 57.7351
[4442]	valid_0's rmse: 57.7357
[4443]	valid_0's rmse: 57.7366
[4444]	valid_0's rmse: 57.7366
[4445]	valid_0's rmse: 57.7381
[4446]	valid_0's rmse: 57.7379
[4447]	valid_0's rmse: 57.7349
[4448]	valid_0's rmse: 57.7342
[4449]	valid_0's rmse: 57.7346
[4450]	valid_0's rmse: 57.732
[4451]	valid_0's rmse: 57.7319
[4452]	valid_0's rmse: 57.732
[4453]	valid_0's rmse: 57.7306
[4454]	valid_0's rmse: 57.7312
[4455]	valid_0's rmse: 57.7313
[4456]	valid_0's rmse: 57.7299
[4457]	valid_0's rmse: 57.7294
[4458]	valid_0's rmse: 57.7293
[4459]	valid_0's rmse: 57.7318
[4460]	valid_0's rmse: 57.7301
[4461]	vali

[4699]	valid_0's rmse: 57.7045
[4700]	valid_0's rmse: 57.7045
[4701]	valid_0's rmse: 57.7046
[4702]	valid_0's rmse: 57.7035
[4703]	valid_0's rmse: 57.7028
[4704]	valid_0's rmse: 57.7034
[4705]	valid_0's rmse: 57.7039
[4706]	valid_0's rmse: 57.7028
[4707]	valid_0's rmse: 57.7027
[4708]	valid_0's rmse: 57.7018
[4709]	valid_0's rmse: 57.6929
[4710]	valid_0's rmse: 57.6914
[4711]	valid_0's rmse: 57.6903
[4712]	valid_0's rmse: 57.6897
[4713]	valid_0's rmse: 57.6892
[4714]	valid_0's rmse: 57.6883
[4715]	valid_0's rmse: 57.6851
[4716]	valid_0's rmse: 57.6856
[4717]	valid_0's rmse: 57.6842
[4718]	valid_0's rmse: 57.6835
[4719]	valid_0's rmse: 57.6833
[4720]	valid_0's rmse: 57.6835
[4721]	valid_0's rmse: 57.6827
[4722]	valid_0's rmse: 57.6831
[4723]	valid_0's rmse: 57.6818
[4724]	valid_0's rmse: 57.6819
[4725]	valid_0's rmse: 57.6819
[4726]	valid_0's rmse: 57.6836
[4727]	valid_0's rmse: 57.6835
[4728]	valid_0's rmse: 57.6825
[4729]	valid_0's rmse: 57.6824
[4730]	valid_0's rmse: 57.6828
[4731]	v

In [349]:
pred = model.predict(sub)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields listing_url, last_scraped, name, summary, space, description, experiences_offered, neighborhood_overview, notes, transit, access, interaction, house_rules, picture_url, host_url, host_name, host_since, host_location, host_about, host_response_time, host_response_rate, host_is_superhost, host_thumbnail_url, host_picture_url, host_neighbourhood, host_verifications, host_has_profile_pic, host_identity_verified, street, neighbourhood, neighbourhood_cleansed, neighbourhood_group_cleansed, city, state, market, smart_location, country_code, country, is_location_exact, property_type, room_type, bed_type, amenities, calendar_updated, has_availability, calendar_last_scraped, first_review, last_review, requires_license, jurisdiction_names, instant_bookable, is_business_travel_ready, cancellation_policy, require_guest_profile_picture, require_guest_phone_verification

In [338]:
sub_id = pd.read_csv('scoringData.csv')

In [343]:
output = pd.DataFrame({'id': sub_id['id'], 'price': pred})

In [344]:
output.to_csv("1.csv", index=False)